In [124]:
import cv2
import os
import numpy as np
import math

In [125]:
def read_images(img_dir):

    # Define the image file extensions to be read
    img_exts = (".jpg", ".jpeg", ".png")

    # Initialize an empty list to store the images
    images = []

    # Loop through all the files in the directory
    for file in sorted(os.listdir(img_dir)):
        # Check if the file has a valid image extension
        if file.lower().endswith(img_exts):
            # Read the image using OpenCV
            img = cv2.imread(os.path.join(img_dir, file))
            # # Convert the image to grayscale and append it to the list
            # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            images.append(img)
            

    # Convert the list of images to a 2D numpy array
    img_array = np.array(images)

    return img_array

In [126]:
def cylindrical_warp(img_array, f):

    proj_images = []

    for img in img_array:
        # Get the dimensions of each image
        # height, width, channels = img.shape
        height, width, channel = img.shape
        # Define the center of the cylindrical coordinates
        x0 = width // 2
        y0 = height // 2

        

        # Create a new image with the same size as the original image
        proj_img = np.zeros_like(img)
        # Iterate over each pixel in the new image
        for i in range(height):
            for j in range(width):
                b=img[i,j,0]
                g=img[i,j,1]
                r=img[i,j,2]
                # Convert the pixel's (x, y) coordinates to cylindrical coordinates (h, theta)
                x = j - x0
                y = i - y0

                h = y/math.sqrt(x**2 + f**2)
                theta = math.atan(x / f)

                # Calculate the new (x, y) coordinates from the cylindrical coordinates using the inverse transform
                
                x_new = f*theta + x0
                y_new = f*h + y0

                # Round the new (x, y) coordinates to the nearest integer to get the corresponding pixel in the original image
                x_new_rd = int(round(x_new))
                y_new_rd = int(round(y_new))

                proj_img[y_new_rd, x_new_rd, 0] = b
                proj_img[y_new_rd, x_new_rd, 1] = g
                proj_img[y_new_rd, x_new_rd, 2] = r


        proj_images.append(proj_img)

    proj_array = np.array(proj_images)

    # cv.imshow("proj_img0", proj_array[0])
    # cv.waitKey(0)
    # cv.destroyAllWindows()

    return proj_array

In [127]:
def feature_detection(img):

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Compute the derivatives of the image intensity in the x and y directions
    dx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    dy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

    # Compute the element-wise product of the derivative images
    Ix2 = dx * dx
    Iy2 = dy * dy
    Ixy = dx * dy

    # Apply a Gaussian filter to each of the derivative images
    ksize = (5, 5)
    sigma = 1.0
    Ix2 = cv2.GaussianBlur(Ix2, ksize, sigma)
    Iy2 = cv2.GaussianBlur(Iy2, ksize, sigma)
    Ixy = cv2.GaussianBlur(Ixy, ksize, sigma)

    # Compute the sums of the products of derivatives at each pixel using a sliding window
    w_size = 5
    offset = w_size // 2
    height, width = gray.shape
    Sxx = np.zeros_like(gray, dtype=np.float32)
    Syy = np.zeros_like(gray, dtype=np.float32)
    Sxy = np.zeros_like(gray, dtype=np.float32)

    for y in range(offset, height - offset):
        for x in range(offset, width - offset):
            Sxx[y, x] = np.sum(Ix2[y - offset:y + offset + 1, x - offset:x + offset + 1])
            Syy[y, x] = np.sum(Iy2[y - offset:y + offset + 1, x - offset:x + offset + 1])
            Sxy[y, x] = np.sum(Ixy[y - offset:y + offset + 1, x - offset:x + offset + 1])

    # Compute the determinant and trace of the structure tensor
    det = (Sxx * Syy) - (Sxy ** 2)
    trace = Sxx + Syy

    # Compute the corner response function
    k = 0.04
    response = det - k * (trace ** 2)

    # Threshold the corner response function
    threshold = 0.01 * response.max()
    corners = np.argwhere(response > threshold)

    # Draw crosses at the detected corners on the image
    for corner in corners:
        x, y = corner
        size = 1
        color = (0, 0, 255)
        cv2.line(img, (y - size, x), (y + size, x), color, 2)
        cv2.line(img, (y, x - size), (y, x + size), color, 2)

    # Show the image with detected corners
    cv2.imshow("Corners Detected", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return corners

In [128]:
import cv2
import numpy as np

def harris_corner_detection(image, block_size=3, ksize=3, k=0.04, threshold=0.06):
    # Convert the input image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Calculate the derivatives of the image using the Sobel operator
    Ix = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=ksize)
    Iy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=ksize)

    # Compute the products of derivatives at each pixel
    Ix2 = Ix ** 2
    Iy2 = Iy ** 2
    Ixy = Ix * Iy

    # Compute the sum of products of derivatives over a local window
    if ksize % 2 == 0 or ksize < 1:
        raise ValueError("Invalid ksize parameter.")
    Sx2 = cv2.boxFilter(Ix2, cv2.CV_64F, (ksize, ksize))
    Sy2 = cv2.boxFilter(Iy2, cv2.CV_64F, (ksize, ksize))
    Sxy = cv2.boxFilter(Ixy, cv2.CV_64F, (ksize, ksize))

    # Compute the Harris response for each pixel
    det = (Sx2 * Sy2) - (Sxy ** 2)
    trace = Sx2 + Sy2
    response = det - k * (trace ** 2)

    # Threshold the response to obtain candidate corners
    response[response < threshold * response.max()] = 0

    # Find the coordinates of the remaining corners
    coords = np.argwhere(response > 0)
    coords = [tuple(coord[::-1]) for coord in coords]

    # Draw circles around the detected corners
    for corner in coords:
        cv2.circle(image, corner, 3, (0, 255, 0), -1)

    # Display the image with detected corners
    cv2.imshow('Harris Corner Detection', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return coords


In [129]:
def feature_descriptor(img, corners):
    patch_size = 16
    subregion_size = 8
    num_bins = 8
    eps = 1e-5

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    height, width = gray.shape

    # Compute the gradient magnitude and orientation for the entire image
    dx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    dy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    mag = np.sqrt(dx**2 + dy**2)
    ang = np.arctan2(dy, dx)

    descriptors = []
    for corner in corners:
        x, y = corner

        # Extract the patch centered at the corner point
        x1 = max(0, x - patch_size//2)
        x2 = min(width-1, x + patch_size//2)
        y1 = max(0, y - patch_size//2)
        y2 = min(height-1, y + patch_size//2)
        patch = mag[y1:y2, x1:x2]
        patch_ang = ang[y1:y2, x1:x2]

        # Divide the patch into subregions and compute histograms of gradient orientations
        desc = []
        for i in range(patch.shape[0]//subregion_size):
            for j in range(patch.shape[1]//subregion_size):
                subpatch = patch[i*subregion_size:(i+1)*subregion_size, j*subregion_size:(j+1)*subregion_size]
                subpatch_ang = patch_ang[i*subregion_size:(i+1)*subregion_size, j*subregion_size:(j+1)*subregion_size]
                hist, _ = np.histogram(subpatch_ang, bins=num_bins, range=(-np.pi, np.pi), weights=subpatch)
                desc.extend(hist)
        
        # Normalize the descriptor to be invariant to changes in illumination, contrast, and scale
        desc = np.array(desc)
        desc = desc / (np.linalg.norm(desc) + eps)

        descriptors.append(desc)
    #print(descriptors[0])

    return descriptors


In [130]:
def feature_matching(desc1, desc2, threshold=0.5):
    matches = []
    for i, d1 in enumerate(desc1):
        best_match = None
        best_distance = float('inf')
        for j, d2 in enumerate(desc2):
            distance = 0
            for k in range(min(len(d1), len(d2))):
                distance += np.sum((d1[k] - d2[k]) ** 2)
            if distance < best_distance:
                best_distance = distance
                best_match = j
        if best_distance < threshold:
            matches.append((i, best_match))

    return matches


In [131]:
def draw_matches(img1, features1, img2, features2, matches):
    matched_img = np.hstack((img1, img2))

    for m in matches:
        pt1 = tuple(map(int, features1[m[0]]))
        pt2 = tuple(map(int, features2[m[1]]))
        cv2.line(matched_img, pt1, (pt2[0] + img1.shape[1], pt2[1]), (0, 255, 0), 1)

    return matched_img

In [132]:
import numpy as np

def find_homography(matches, k):
    """
    Finds the homography matrix using Direct Linear Transformation (DLT) algorithm.
    Inputs:
        - matches: a list of matching feature points, each point is a tuple of (x, y) coordinates.
        - k: number of iterations for RANSAC algorithm.
    Outputs:
        - H: the homography matrix.
    """
    # Convert the feature points to homogeneous coordinates
    src_pts = np.ones((len(matches), 3))
    dst_pts = np.ones((len(matches), 3))
    for i, match in enumerate(matches):
        src_pts[i, :2] = match[0]
        dst_pts[i, :2] = match[1]

    # Normalize the coordinates to improve numerical stability
    src_pts[:, :2] /= src_pts[:, 2, None]
    dst_pts[:, :2] /= dst_pts[:, 2, None]

    # RANSAC algorithm to remove outliers
    best_inliers = []
    for i in range(k):
        # Randomly select four pairs of matching points
        indices = np.random.choice(len(matches), 4, replace=False)
        src = src_pts[indices]
        dst = dst_pts[indices]

        # Compute the homography matrix using DLT algorithm
        A = np.zeros((8, 9))
        for j in range(4):
            A[j*2, :3] = src[j]
            A[j*2, 6:] = -dst[j, 0] * src[j]
            A[j*2+1, 3:6] = src[j]
            A[j*2+1, 6:] = -dst[j, 1] * src[j]
        _, _, V = np.linalg.svd(A)
        H = V[-1, :].reshape((3, 3))

        # Compute the inliers
        inliers = []
        for j, match in enumerate(matches):
            x1 = src_pts[j]
            x2 = dst_pts[j]
            x2_hat = H @ x1
            x2_hat /= x2_hat[2]
            if np.linalg.norm(x2 - x2_hat) < 5:
                inliers.append(j)

        # Update the best set of inliers
        if len(inliers) > len(best_inliers):
            best_inliers = inliers

    # Refit the homography matrix using all inliers
    src = src_pts[best_inliers]
    dst = dst_pts[best_inliers]
    A = np.zeros((2*len(src), 9))
    for i in range(len(src)):
        A[2*i, :3] = src[i]
        A[2*i, 6:] = -dst[i, 0] * src[i]
        A[2*i+1, 3:6] = src[i]
        A[2*i+1, 6:] = -dst[i, 1] * src[i]
    _, _, V = np.linalg.svd(A)
    H = V[-1, :].reshape((3, 3))

    return H

In [133]:
def good_matching(matches, homography_matrix, threshold=5.0):
    good_matches = []
    for match in matches:
        pt1 = np.array([match[0][0], match[0][1], 1]).reshape(-1, 1)
        pt2 = np.array([match[1][0], match[1][1], 1]).reshape(-1, 1)
        transformed_pt1 = homography_matrix @ pt1
        transformed_pt1 /= transformed_pt1[2, 0]
        distance = np.linalg.norm(transformed_pt1[:2, 0] - pt2[:2, 0])
        if distance < threshold:
            good_matches.append(match)
    return good_matches

In [134]:
def stitch_images(img1, img2, matches, H):
    # Get the size of the output image
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    corners = np.array([[0, 0, 1], [0, h2-1, 1], [w2-1, h2-1, 1], [w2-1, 0, 1]])
    transformed_corners = np.dot(corners, H.T)
    transformed_corners[:, 0] /= transformed_corners[:, 2]
    transformed_corners[:, 1] /= transformed_corners[:, 2]
    x_min = int(min(np.min(transformed_corners[:, 0]), 0))
    y_min = int(min(np.min(transformed_corners[:, 1]), 0))
    x_max = int(max(np.max(transformed_corners[:, 0]), w1))
    y_max = int(max(np.max(transformed_corners[:, 1]), h1))
    output_size = (x_max - x_min, y_max - y_min)

    # Compute the translation matrix
    T = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]])

    # Create an empty output image
    output = np.zeros((output_size[1], output_size[0], 3), dtype=np.uint8)

    # Copy the first image onto the output image
    output[-y_min:h1-y_min, -x_min:w1-x_min] = img1

    # Warp the second image using the homography matrix
    warped = cv2.warpPerspective(img2, T.dot(H), output_size)

    # Copy the warped second image onto the output image
    for i, match in enumerate(matches):
        pt1 = np.array([int(match[0][0]), int(match[0][1])])
        pt2 = np.array([int(match[1][0] - x_min), int(match[1][1] - y_min)])
        output = cv2.line(output, tuple(pt1), tuple(pt2), (0, 255, 0), 1)
    for y in range(output_size[1]):
        for x in range(output_size[0]):
            if np.any(warped[y, x]):
                output[y, x] = warped[y, x]

    return output


In [135]:
if __name__ == '__main__':
    img_array = read_images("test_images")
    proj_array = cylindrical_warp(img_array, 900)
    corners0 = harris_corner_detection(proj_array[0])
    corners1 = harris_corner_detection(proj_array[1])
    descriptors0 = feature_descriptor(proj_array[0], corners0)
    descriptors1 = feature_descriptor(proj_array[1], corners1)
    matches = feature_matching(descriptors0, descriptors1)

    matched_img = draw_matches(proj_array[0], corners0, proj_array[1], corners1, matches)
    cv2.imshow('matched_img', matched_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    h = find_homography(matches, 1000)
    result = stitch_images(proj_array[0], proj_array[1], matches, h)
    cv2.imshow('warped_img', result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    


TypeError: 'int' object is not subscriptable